In [100]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import sqlite3

In [60]:
def extract(url, tablecontent):
    page = requests.get(url).text
    Data = BeautifulSoup(page, 'html.parser')
    Tables = Data.find_all('table')
    rows = Tables[0].find_all('tr')
    df = pd.DataFrame(columns = tablecontent)

    for row in rows:
        cols = row.find_all('td')
        if len(cols)!=0:
            if cols[1].find('a') is not None:
                data_dict = {'Bank Name': cols[1].find_all('a')[1].contents[0],
                             'MC_USD_Billion': cols[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df1,df], ignore_index= True)
    return df    

In [85]:
def transform(df):
    mc_list = df['MC_USD_Billion'].tolist()
    mc_list = [float("".join(x.split('\\'))) for x in mc_list]
    df['MC_USD_Billion'] = mc_list
    df['MC_GBP_Billion'] = round((df['MC_USD_Billion'] / 0.93),2)
    df['MC_EUR_Billion'] = round((df['MC_USD_Billion'] / 0.8),2)
    df['MC_INR_Billion'] = round((df['MC_USD_Billion'] / 82.95),2)
    return df

In [98]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)

In [91]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

In [92]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [93]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

In [110]:
url = 'https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'
tablecontent = ['Bank Name', 'MC_USD_Billion']
csv_path = 'Largest Banks Data.csv'
table_name = 'Largest_Banks'

In [111]:
log_progress('Preliminaries complete. Initiating ETL process')
df = extract(url, tablecontent)
log_progress('Data extraction complete. Initiating Transformation process')
df = transform(df)
log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')
sql_connection = sqlite3.connect('Banks.db')
log_progress('SQL Connection initiated.')
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query')
query_statement = f"SELECT * from {table_name}"
run_query(query_statement, sql_connection)
log_progress('Process Complete.')
sql_connection.close()

SELECT *from Largest_Banks
                                 Bank Name  MC_USD_Billion  MC_GBP_Billion  \
0                            Bank of China          136.81          147.11   
1                  China Construction Bank          139.82          150.34   
2                           Morgan Stanley          140.83          151.43   
3                        HSBC Holdings PLC          148.90          160.11   
4                              Wells Fargo          155.87          167.60   
5                                HDFC Bank          157.91          169.80   
6               Agricultural Bank of China          160.68          172.77   
7  Industrial and Commercial Bank of China          194.56          209.20   
8                          Bank of America          231.52          248.95   
9                           JPMorgan Chase          432.92          465.51   

   MC_EUR_Billion  MC_INR_Billion  
0          171.01            1.65  
1          174.77            1.69  
2     